In [ ]:
#!pip install astropy==5.1.1 --no-cache-dir --force-reinstall
#!pip install astropy==4.3.1 --no-cache-dir --force-reinstall
!pip install astropy==5.1.1 

In [ ]:
#!pip install aplpy --no-cache-dir --force-reinstall
!pip install aplpy

In [8]:
import matplotlib.pyplot as plt
import aplpy
import os

In [14]:
import warnings
warnings.filterwarnings("ignore")
from astropy.io import fits

In [ ]:
path = '../data/new_data_subset/cut_fits/'
destination = '../data/new_data_subset/masks_from_cut_fits/images/'

def convert_data():
    i = 0

    for index, file in enumerate(os.listdir(path)):
        file_name = file.split(".")[0]


        #gc = aplpy.FITSFigure(path + file).show_grayscale(invert=False, stretch='power', exponent=0.5,pmin=10,pmax=90)
        gc = aplpy.FITSFigure(path + file).show_grayscale(invert=False, stretch='power', exponent=0.5)
        plt.gca().set_axis_off()
        plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                            hspace=0, wspace=0)
        plt.margins(0, 0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())

        plt.savefig(destination + '{}.jpg'.format(file_name), format='jpg', bbox_inches='tight', pad_inches=0)
        
convert_data()

In [5]:
import logging

logging.disable(logging.INFO)

In [7]:
from concurrent.futures import ProcessPoolExecutor
batch_number = 28
path = f'/home/jovyan/data2/uniba/sdss_source_fits/folder_{batch_number}/'
destination = f'/home/jovyan/data2/uniba/sdss_source_jpg/folder_{batch_number}/'


def process_file(file):
    file_name = file.split(".")[0]
    gc = aplpy.FITSFigure(path + file).show_grayscale(invert=False, stretch='power', exponent=0.5)

    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())

    plt.savefig(destination + '{}.jpg'.format(file_name), format='jpg', bbox_inches='tight', pad_inches=0)
    plt.close()


def convert_data():
    if not os.path.exists(destination):
        os.makedirs(destination)
    files = os.listdir(path)
    with ProcessPoolExecutor() as executor:
        executor.map(process_file, files)


convert_data()

### multiprocessing wrong 

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

import aplpy
from matplotlib import pyplot as plt


def process_file(file, path, destination):
    file_name = file.split(".")[0]
    gc = aplpy.FITSFigure(path + file).show_grayscale(invert=False, stretch='power', exponent=0.5)

    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())

    plt.savefig(destination + '{}.jpg'.format(file_name), format='jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
a

def convert_data(batch_number):

    path = f'/home/jovyan/data2/uniba/sdss_source_fits/folder_{batch_number}/'
    destination = f'/home/jovyan/data2/uniba/sdss_source_jpg/folder_{batch_number}/'

    if not os.path.exists(destination):
        os.makedirs(destination)
    files = os.listdir(path)
    with ThreadPoolExecutor(max_workers=24) as executor:
        futures = [executor.submit(process_file, file, path, destination) for file in files]

        for future in as_completed(futures):
            try:
                galaxy_data = future.result()
                df_galaxy_data = df_galaxy_data.append(galaxy_data.__dict__, ignore_index=True)
            except Exception as e:
                print(f"Error processing FIT: {e}")


if __name__ == "__main__":
    for batch_number in range(3, 29):
        convert_data(batch_number)


### Pixel dump instead of matplotlib

In [ ]:
from astropy.io import fits
from PIL import Image
import numpy as np
import os

# Paths
fits_dir = '../data/new_data_subset/cut_fits/'
destination = '../data/new_data_subset/masks_from_cut_fits/images_dump/'

os.makedirs(destination, exist_ok=True)

def convert_fits_to_exact_image():
    for file in os.listdir(fits_dir):
        file_path = os.path.join(fits_dir, file)
        file_name = file.split(".")[0]

        # Load FITS data
        with fits.open(file_path) as hdul:
            data = hdul[0].data

        # Flip vertically to match FITS (bottom-left) to image (top-left)
        data = data[::-1]

        # Apply power stretch (same as aplpy stretch='power', exponent=0.5)
        exponent = 0.5
        data = np.power(data - np.min(data), exponent)

        # Normalize to 0-255 (8-bit grayscale)
        data -= np.min(data)
        data /= np.max(data)
        data *= 255
        data = data.astype(np.uint8)

        # Save as image
        image = Image.fromarray(data)
        image.save(os.path.join(destination, f'{file_name}.png'))

        #print(f"Converted {file} to {file_name}.png with power stretch (exponent={exponent})")

convert_fits_to_exact_image()